# 01 — Classical Baseline for Binary Classification

This notebook establishes the **performance benchmark** using a pure PyTorch MLP
on the Breast Cancer Wisconsin dataset. The results here serve as the reference
against which our hybrid quantum-classical models will be compared.

In [ ]:
import sys
from pathlib import Path

# Project root
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid', palette='colorblind')

from src.utils.config_loader import load_config, get_device
from src.utils.quantum_utils import set_seed, count_model_parameters
from src.data.dataloaders import get_data_from_config
from src.models.classical_net import ClassicalNet
from src.training.trainer import Trainer
from src.evaluation.metrics import evaluate_model, compute_confusion_matrix
from src.evaluation.visualization import plot_training_curves, plot_confusion_matrix

print(f'PyTorch {torch.__version__}')
print(f'Device: {get_device()}')

## 1. Load Configuration & Data

In [ ]:
cfg = load_config(PROJECT_ROOT / 'config' / 'classical_baseline.yaml')
set_seed(cfg['experiment']['seed'])

train_loader, test_loader, input_dim = get_data_from_config(cfg, PROJECT_ROOT)
print(f'Input features: {input_dim}')
print(f'Train batches: {len(train_loader)}, Test batches: {len(test_loader)}')

## 2. Build Model

In [ ]:
cfg['classical']['input_dim'] = input_dim
model = ClassicalNet.from_config(cfg['classical'])

params = count_model_parameters(model)
print(f'Architecture: {model}')
print(f'\nParameter count: {params}')

## 3. Train

In [ ]:
device = get_device(cfg['experiment'].get('device', 'auto'))
model = model.to(device)

trainer = Trainer(model, cfg)
history = trainer.train(train_loader, test_loader)

## 4. Training Curves

In [ ]:
fig = plot_training_curves(history, title='Classical Baseline — Training History')
plt.show()

## 5. Evaluation & Confusion Matrix

In [ ]:
metrics = evaluate_model(
    model, test_loader, device=str(device),
    metric_names=['accuracy', 'f1', 'roc_auc', 'precision', 'recall']
)
print('\n=== Test Set Metrics ===')
for k, v in metrics.items():
    print(f'  {k:15s}: {v:.4f}')

# Confusion matrix
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for X, y in test_loader:
        preds = model(X.to(device)).argmax(dim=1).cpu()
        all_preds.append(preds)
        all_labels.append(y)
y_pred = torch.cat(all_preds).numpy()
y_true = torch.cat(all_labels).numpy()
cm = compute_confusion_matrix(y_true, y_pred)

fig = plot_confusion_matrix(cm, class_names=['Malignant', 'Benign'],
                            title='Classical Baseline — Confusion Matrix')
plt.show()